In [5]:
%matplotlib qt
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import numpy.linalg as la
from sympy import Matrix,symbols,solve

In [6]:
def get_input_lines(im, min_lines=3):
    """
    Allows user to input line segments; computes centers and directions.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        min_lines: minimum number of lines required
    Returns:
        n: number of lines from input
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        centers: np.ndarray of shape (3, n)
            where each column denotes the homogeneous coordinates of the centers
    """
    n = 0
    lines = np.zeros((3, 0))
    centers = np.zeros((3, 0))

    plt.figure()
    plt.imshow(im)
    plt.show()
    print('Set at least %d lines to compute vanishing point' % min_lines)
    while True:
        print('Click the two endpoints, use the right key to undo, and use the middle key to stop input')
        clicked = plt.ginput(2, timeout=0, show_clicks=True)
        if not clicked or len(clicked) < 2:
            if n < min_lines:
                print('Need at least %d lines, you have %d now' % (min_lines, n))
                continue
            else:
                # Stop getting lines if number of lines is enough
                break

        # Unpack user inputs and save as homogeneous coordinates
        pt1 = np.array([clicked[0][0], clicked[0][1], 1])
        pt2 = np.array([clicked[1][0], clicked[1][1], 1])
        # Get line equation using cross product
        # Line equation: line[0] * x + line[1] * y + line[2] = 0
        line = np.cross(pt1, pt2)
        lines = np.append(lines, line.reshape((3, 1)), axis=1)
        # Get center coordinate of the line segment
        center = (pt1 + pt2) / 2
        centers = np.append(centers, center.reshape((3, 1)), axis=1)

        # Plot line segment
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], color='b')

        n += 1

    return n, lines, centers


def plot_lines_and_vp(im, lines, vp):
    """
    Plots user-input lines and the calculated vanishing point.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        lines: np.ndarray of shape (3, n)
            where each column denotes the parameters of the line equation
        vp: np.ndarray of shape (3, )
    """
    bx1 = min(1, vp[0] / vp[2]) - 10
    bx2 = max(im.shape[1], vp[0] / vp[2]) + 10
    by1 = min(1, vp[1] / vp[2]) - 10
    by2 = max(im.shape[0], vp[1] / vp[2]) + 10

    plt.figure()
    plt.imshow(im)
    for i in range(lines.shape[1]):
        if lines[0, i] < lines[1, i]:
            pt1 = np.cross(np.array([1, 0, -bx1]), lines[:, i])
            pt2 = np.cross(np.array([1, 0, -bx2]), lines[:, i])
        else:
            pt1 = np.cross(np.array([0, 1, -by1]), lines[:, i])
            pt2 = np.cross(np.array([0, 1, -by2]), lines[:, i])
        pt1 = pt1 / pt1[2]
        pt2 = pt2 / pt2[2]
        plt.plot([pt1[0], pt2[0]], [pt1[1], pt2[1]], 'g')

    plt.plot(vp[0] / vp[2], vp[1] / vp[2], 'ro')
    plt.show()




def get_top_and_bottom_coordinates(im, obj):
    """
    For a specific object, prompts user to record the top coordinate and the bottom coordinate in the image.
    Inputs:
        im: np.ndarray of shape (height, width, 3)
        obj: string, object name
    Returns:
        coord: np.ndarray of shape (3, 2)
            where coord[:, 0] is the homogeneous coordinate of the top of the object and coord[:, 1] is the homogeneous
            coordinate of the bottom
    """
    plt.figure()
    plt.imshow(im)

    print('Click on the top coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x1, y1 = clicked[0]
    # Uncomment this line to enable a vertical line to help align the two coordinates
    # plt.plot([x1, x1], [0, im.shape[0]], 'b')
    print('Click on the bottom coordinate of %s' % obj)
    clicked = plt.ginput(1, timeout=0, show_clicks=True)
    x2, y2 = clicked[0]

    plt.plot([x1, x2], [y1, y2], 'b')

    return np.array([[x1, x2], [y1,y2],[1,1]])

In [ ]:
def get_vanishing_point(lines):
    """
    Solves for the vanishing point using the user-input lines.
    """
    n = lines.shape[1]

    intersections_s = []
    for i in range(n):
        for j in range(i + 1, n):  
            intersection = np.cross(lines[:, i], lines[:, j])
            intersections_s.append(intersection)
    
    intersections_s = np.array(intersections_s)
    #print(intersections_s)
    vanishing_point = intersections_s.mean(axis=0)
    return vanishing_point


def get_horizon_line(vanishing_point):
    """
    Calculates the ground horizon line.
    """
    vp_left = vanishing_point[:, 0]
    vp_right = vanishing_point[:, 2]
    horizon_line = np.cross(vp_left, vp_right)
    normalization_factor = la.norm(horizon_line[:2])
    horizon_line = horizon_line / normalization_factor

    return horizon_line



def plot_horizon_line(im , horizon_line):
    """
    Plots the horizon line.
    """
    plt.figure()
    plt.imshow(im)
    plt.show()

    a, b, c = horizon_line

    height, width, channel = im.shape

    x_coords = np.linspace(0, width, num=2)

    y_coords = (-c - a * x_coords) / b

    print(f"Y-coordinates for horizon line: {y_coords}")
    print(a,b,c)

    plt.plot(x_coords, y_coords, color='blue')  # Added color for better visibility
    plt.show()


def get_camera_parameters(vanishing_pts):
    """
    Computes the camera intrinsic parameters using SymPy.
    Inputs:
        vpts: A NumPy array of shape (3, 3) where each column is a vanishing point in homogeneous coordinates.
    Outputs:
        f: Focal length
        u: Principal point x-coordinate
        v: Principal point y-coordinate
    """
    v_point_vertical = Matrix(vanishing_pts[:, 0])  
    v_point_right = Matrix(vanishing_pts[:, 1])     
    v_point_left = Matrix(vanishing_pts[:, 2])      

    focal_len, principal_x, principal_y = symbols('focal_len principal_x principal_y')

    inverse_intrinsic = Matrix([
        [1 / focal_len, 0, -principal_x / focal_len],
        [0, 1 / focal_len, -principal_y / focal_len],
        [0, 0, 1]
    ])

    constraint_1 = (v_point_vertical.T * inverse_intrinsic.T * inverse_intrinsic * v_point_right)[0]
    constraint_2 = (v_point_vertical.T * inverse_intrinsic.T * inverse_intrinsic * v_point_left)[0]
    constraint_3 = (v_point_right.T * inverse_intrinsic.T * inverse_intrinsic * v_point_left)[0]

    solutions = solve([constraint_1, constraint_2, constraint_3], (focal_len, principal_x, principal_y), dict=True)

    if not solutions:
        print("No symbolic solution found. Falling back to approximation.")
        px_estimate = np.mean(vanishing_pts[0, :])  
        py_estimate = np.mean(vanishing_pts[1, :])  
        focal_estimate = np.sqrt(
            (vanishing_pts[0, 0] - px_estimate) ** 2 + (vanishing_pts[1, 0] - py_estimate) ** 2
        )
        return abs(focal_estimate), px_estimate, py_estimate

    f_result = solutions[0][focal_len]
    px_result = solutions[0][principal_x]
    py_result = solutions[0][principal_y]

    return abs(f_result), px_result, py_result
def get_rotation_matrix(focal_length, principal_x, principal_y, vanishing_points):

    """
    Computes the rotation matrix using the camera parameters.
    """
    
    focal_length = float(focal_length)
    principal_x = float(principal_x)
    principal_y = float(principal_y)

    intrinsic_matrix = np.array([
        [focal_length, 0, principal_x],
        [0, focal_length, principal_y],
        [0, 0, 1]
    ])

    # Map vanishing points to the camera coordinate system
    rotation = la.inv(intrinsic_matrix) @ vanishing_points

    # Normalize each column to ensure orthonormal vectors
    for i in range(rotation.shape[1]):
        rotation[:, i] /= np.linalg.norm(rotation[:, i])

    return rotation

def estimate_height(ref_points, vanishing_pts, obj_points, ref_height):

    """
    Estimates height for a specific object using the recorded coordinates.
    """
    horizon_line = np.cross(vanishing_pts[:, 2], vanishing_pts[:, 0])

    obj_ref_line = np.cross(obj_points[:, 1], ref_points[:, 1])
    horizon_intersection = np.cross(horizon_line, obj_ref_line)
    horizon_intersection /= horizon_intersection[2]

    ref_bottom_line = np.cross(horizon_intersection, ref_points[:, 0])

    obj_vertical_line = np.cross(obj_points[:, 0], obj_points[:, 1])
    proportional_point = np.cross(ref_bottom_line, obj_vertical_line)
    proportional_point /= proportional_point[2]

    top_to_proportional = la.norm(proportional_point - obj_points[:, 1])
    bottom_to_top = la.norm(obj_points[:, 0] - obj_points[:, 1])
    vertical_to_bottom = la.norm(vanishing_pts[:, 1] - obj_points[:, 0])
    vertical_to_proportional = la.norm(vanishing_pts[:, 1] - proportional_point)

    height = ref_height * (bottom_to_top * vertical_to_proportional) / (top_to_proportional * vertical_to_bottom)
    print(height)
    return abs(height)

In [8]:
im = np.asarray(Image.open(r'ECEB.jpg'))

In [9]:
#part1
# Get vanishing points for each of the directions
num_vpts = 3
vpts = np.zeros((3, num_vpts))
for i in range(num_vpts):
    print('Getting vanishing point %d' % i)
    n, lines, centers = get_input_lines(im)
    vpts[:, i] = get_vanishing_point(lines)
    plot_lines_and_vp(im, lines, vpts[:, i])
print(vpts)
horizon_line = get_horizon_line(vpts)
plot_horizon_line(im,horizon_line)

Getting vanishing point 0
Set at least 3 lines to compute vanishing point
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Getting vanishing point 1
Set at least 3 lines to compute vanishing point
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Click the two endpoints, use the right key to undo, and use the middle key to stop input
Getting vanishing point 2
Set at least 3 lines to compute vanishing point
Click the two endpoints, use the right key to undo, and use the mi

<ipython-input-6-10c44840881c>:75: RuntimeWarning: divide by zero encountered in true_divide
  pt1 = pt1 / pt1[2]
<ipython-input-6-10c44840881c>:75: RuntimeWarning: invalid value encountered in true_divide
  pt1 = pt1 / pt1[2]
<ipython-input-6-10c44840881c>:76: RuntimeWarning: divide by zero encountered in true_divide
  pt2 = pt2 / pt2[2]
<ipython-input-6-10c44840881c>:76: RuntimeWarning: invalid value encountered in true_divide
  pt2 = pt2 / pt2[2]


In [17]:
# Part 2
f, u, v = get_camera_parameters(vpts)

# Part 3
R = get_rotation_matrix(f,u,v,vpts)

# Part 4
objects = ('person', 'ECE left building', 'ECE Right building', 'the lamp posts')
reference_height = 1.83
coords = dict()
for obj in objects:
    coords[obj] = get_top_and_bottom_coordinates(im, obj)

for obj in objects:
    print('Estimating height of %s' % obj)
    height = estimate_height(coords['person'], vpts, coords[obj], reference_height)

Click on the top coordinate of person
Click on the bottom coordinate of person
Click on the top coordinate of ECE left building
Click on the bottom coordinate of ECE left building
Click on the top coordinate of ECE Right building
Click on the bottom coordinate of ECE Right building
Click on the top coordinate of the lamp posts
Click on the bottom coordinate of the lamp posts
Estimating height of person
nan
Estimating height of ECE left building
23.290322329879373
Estimating height of ECE Right building
10.855536740386402
Estimating height of the lamp posts
2.905856230310703


<ipython-input-7-35945313365b>:167: RuntimeWarning: invalid value encountered in true_divide
  horizon_intersection /= horizon_intersection[2]
